In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import year, month, sum


In [0]:
spark = SparkSession.builder.appName("Goldapp").getOrCreate()
silver_df = spark.read.table("workspace.expense_analytics_silver.expenses_clean")


#Monthly Expense Summary

In [0]:
monthly_summary = (
    silver_df
    .withColumn("year", year("date"))
    .withColumn("month", month("date"))
    .groupBy("year", "month")
    .agg(sum("amount").alias("total_expense"))
)

monthly_summary.write.mode("overwrite").saveAsTable(
    "workspace.expense_analytics_gold.gold_monthly_expense"
)

monthly_summary.show()

## Category Summary

In [0]:
category_summary = silver_df.groupBy("category").agg(sum("amount").alias("total_spent"))

category_summary.write.mode("overwrite").saveAsTable("workspace.expense_analytics_gold.gold_category_summary")

## Payment Mode Analysis

In [0]:
payment_mode_summary = (
    silver_df
    .groupBy("payment_mode")
    .agg(sum("amount").alias("total_spent"))
)

payment_mode_summary.write.mode("overwrite").saveAsTable("workspace.expense_analytics_gold.gold_payment_mode_summary")

## ML Feature Table

In [0]:
ml_features = (
    silver_df
    .withColumn("year", year("date"))
    .withColumn("month", month("date"))
    .groupBy("year", "month", "category")
    .agg(sum("amount").alias("monthly_category_spent"))
)

ml_features.write.mode("overwrite").saveAsTable("workspace.expense_analytics_gold.gold_ml_features")